Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved. This source code is licensed under the license found in the LICENSE file in the root directory of this source tree.

# Video Seal Inference

[[`arXiv`](https://arxiv.org/abs/2412.09492)]
[[`Colab`](https://colab.research.google.com/github/facebookresearch/videoseal/blob/main/notebooks/colab.ipynb)]
[[`Demo`](https://aidemos.meta.com/videoseal)]

## Installation

In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import time

# Create the base directory for video datasets
!mkdir -p /content/sa-v/train
!mkdir -p /content/sa-v/validate
!mkdir -p /content/sa-v/test


# Create directories for SA-1B dataset (COCO)
!mkdir -p /content/sa-1b/train
!mkdir -p /content/sa-1b/validate
!mkdir -p /content/sa-1b/annotations



time.sleep(1)

In [3]:
!pip install kaggle


In [4]:
from google.colab import files
files.upload()


{}

In [5]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [6]:
import time

# Download a subset of the dataset directly into the train, validate, and test directories
!cd /content/sa-v/train && curl --insecure https://data.yt8m.org/download.py | shard=1,100 partition=2/frame/train mirror=eu python
!cd /content/sa-v/validate && curl --insecure https://data.yt8m.org/download.py | shard=1,100 partition=2/frame/validate mirror=eu python
!cd /content/sa-v/test && curl --insecure https://data.yt8m.org/download.py | shard=1,100 partition=2/frame/test mirror=eu python

print("Download complete. Data saved in /content/sa-v/")

time.sleep(5)

!kaggle datasets download -d nhphucqt/coco2017-person-lite -p /content/sa-1b/

time.sleep(2)

!unzip /content/sa-1b/coco2017-person-lite.zip -d /content/sa-1b/


Streaming output truncated to the last 5000 lines.
  inflating: /content/sa-1b/train2017/000000540457.jpg  
  inflating: /content/sa-1b/train2017/000000540469.jpg  
  inflating: /content/sa-1b/train2017/000000540471.jpg  
  inflating: /content/sa-1b/train2017/000000540473.jpg  
  inflating: /content/sa-1b/train2017/000000540486.jpg  
  inflating: /content/sa-1b/train2017/000000540495.jpg  
  inflating: /content/sa-1b/train2017/000000540520.jpg  
  inflating: /content/sa-1b/train2017/000000540561.jpg  
  inflating: /content/sa-1b/train2017/000000540575.jpg  
  inflating: /content/sa-1b/train2017/000000540577.jpg  
  inflating: /content/sa-1b/train2017/000000540614.jpg  
  inflating: /content/sa-1b/train2017/000000540645.jpg  
  inflating: /content/sa-1b/train2017/000000540670.jpg  
  inflating: /content/sa-1b/train2017/000000540716.jpg  
  inflating: /content/sa-1b/train2017/000000540733.jpg  
  inflating: /content/sa-1b/train2017/000000540783.jpg  
  inflating: /content/sa-1b/train2017

In [7]:
import glob

for split in ["train", "validate", "test"]:
    tfrecord_files = glob.glob(f"/content/sa-v/{split}/*.tfrecord")
    print(f"Found {len(tfrecord_files)} TFRecord files in /content/sa-v/{split}/")


Found 41 TFRecord files in /content/sa-v/train/
Found 39 TFRecord files in /content/sa-v/validate/
Found 40 TFRecord files in /content/sa-v/test/


In [8]:
import uuid
import time
import pandas as pd
from google.colab import drive

# Set output file path in Google Drive
output_file = "/content/drive/MyDrive/uuid_timestamps.csv"

# Function to generate UUID.timestamp.fileID
def generate_uuid_timestamp_fileid(num_samples=2000):
    data = []
    for i in range(num_samples):
        uuid_val = uuid.uuid4()
        timestamp = int(time.time())
        file_id = f"file_{i}"
        unique_str = f"{uuid_val}.{timestamp}.{file_id}"
        data.append(unique_str)
    return data

# Generate 2000 sample UUIDs
uuids_list = generate_uuid_timestamp_fileid()

# Save to CSV
df = pd.DataFrame(uuids_list, columns=['uuid_timestamp_fileID'])
df.to_csv(output_file, index=False)

print(f"UUIDs saved to {output_file}")


UUIDs saved to /content/drive/MyDrive/uuid_timestamps.csv


Clone repository and install dependencies

In [9]:
!git clone https://github.com/Puddings22/waterseal.git
%cd /content/waterseal/videoseal-main

Cloning into 'waterseal'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 156 (delta 33), reused 143 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (156/156), 25.09 MiB | 32.04 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/waterseal/videoseal-main


Install dependencies

In [10]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.9 MB/s eta 0:00:00
  Created whee

## Imports and loading

In [11]:
%cd /content/waterseal/videoseal-main

/content/waterseal/videoseal-main


In [12]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import logging
logging.getLogger("matplotlib.image").setLevel(logging.ERROR)
from IPython.display import HTML, display

import pandas as pd
from tqdm import tqdm
import numpy as np
import ffmpeg
import os
import cv2
import subprocess

import torch

from videoseal.evals.metrics import bit_accuracy
from videoseal.models import Videoseal
from videoseal.utils.cfg import setup_model_from_model_card


def get_video_info(input_path):
    # Open the video file
    video = cv2.VideoCapture(input_path)

    # Get video properties
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    codec = int(video.get(cv2.CAP_PROP_FOURCC))
    num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Decode codec to human-readable form
    codec_str = "".join([chr((codec >> 8 * i) & 0xFF) for i in range(4)])

    video.release()  # Close the video file

    return {
        "width": width,
        "height": height,
        "fps": fps,
        "codec": codec_str,
        "num_frames": num_frames
    }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load the model

The videoseal library provides pretrained models for embedding and extracting watermarks.

In [13]:
# Load the VideoSeal model
model = setup_model_from_model_card("videoseal")

# Set the model to evaluation mode and move it to the selected device
model = model.eval()
model = model.to(device)
model.compile()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


checkpoint.pth:   0%|          | 0.00/165M [00:00<?, ?B/s]

Model loaded successfully from /root/.cache/huggingface/hub/models--facebook--video_seal/snapshots/8037ef59ba2b2ec8fb8b55298ff37b8ccddd078d/checkpoint.pth with message: <All keys matched successfully>


In [18]:
!pip install git+https://github.com/vahidk/tfrecord.git

  Cloning https://github.com/vahidk/tfrecord.git to /tmp/pip-req-build-x2p86v1r
  Running command git clone --filter=blob:none --quiet https://github.com/vahidk/tfrecord.git /tmp/pip-req-build-x2p86v1r
  Resolved https://github.com/vahidk/tfrecord.git to commit 0a412c636a8d04ab5a89b443b3ebfa2a7b358202
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.1 MB/s eta 0:00:00
  Created wheel for tfrecord: filename=tfrecord-1.14.5-py3-none-any.whl size=14817 sha256=6c2e856f9dbfaf662eaab14564a64c3f21c2660f0d574ba8e99b418f3cd9f009
  Stored in directory: /tmp/pip-ephem-wheel-cache-sr7tvknb/wheels/09/63/09/a5eb445a02b018d94f42190f35c72a77512b8f01cf44230a69
Successfully built tfrecord


In [21]:
#TEST BEFORE TRAINING

from videoseal.data.loader import get_tf-record_dataloader
train_loader = get_tfrecord_dataloader("/content/sa-v/train", batch_size=2)
for batch_idx, (videos, labels) in enumerate(train_loader):
    print(f"Batch {batch_idx}: videos shape = {videos.shape}, labels shape = {labels.shape}")
    break


SyntaxError: invalid syntax (<ipython-input-21-094993fdc6b8>, line 3)

In [20]:
!python train.py \
    --image_dataset sa-1b \
    --video_dataset sa-v \
    --nbits 384 \
    --img_size 256 \
    --batch_size 8 \
    --epochs 100 \
    --output_dir /content/output/ \
    --workers 4 \
    --embedder_config configs/embedder.yaml \
    --extractor_config configs/extractor.yaml \
    --attenuation_config configs/attenuation.yaml \
    --only_eval False \
    --saveckpt_freq 5 \
    --resume_from /content/output/checkpoint.pth \
    --batch_size_eval 8 \
    --sleepwake False \
    --scaling_w 0.2 \
    --scaling_i 1.0 \
    --optimizer "AdamW,lr=1e-4" \
    --scheduler "StepLR,step_size=30,gamma=0.1"


2025-01-20 19:18:36.078884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-20 19:18:36.098389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-20 19:18:36.104407: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 19:18:37.405844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Traceback (most recent call last):
  File "/content/waterseal/videoseal-main/train.py", line 42, in <module>
    from videoseal.data.loader import (
  File "/content/waterseal/videoseal-mai

## Embedding

The embedding process is the process of hiding the watermark in the video.

In [ ]:
def embed_video_clip(
    model: Videoseal,
    clip: np.ndarray,
    msgs: torch.Tensor
) -> np.ndarray:
    clip_tensor = torch.tensor(clip, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
    outputs = model.embed(clip_tensor, msgs=msgs, is_video=True)
    processed_clip = outputs["imgs_w"]
    processed_clip = (processed_clip * 255.0).byte().permute(0, 2, 3, 1).numpy()
    return processed_clip

def embed_video(
    model: Videoseal,
    input_path: str,
    output_path: str,
    chunk_size: int,
    crf: int = 23
) -> None:
    # Read video dimensions
    video_info = get_video_info(input_path)
    width = int(video_info['width'])
    height = int(video_info['height'])
    fps = float(video_info['fps'])
    codec = video_info['codec']
    num_frames = int(video_info['num_frames'])

    # Open the input video
    process1 = (
        ffmpeg
        .input(input_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height), r=fps)
        .run_async(pipe_stdout=True, pipe_stderr=subprocess.PIPE)
    )
    # Open the output video
    process2 = (
        ffmpeg
        .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height), r=fps)
        .output(output_path, vcodec='libx264', pix_fmt='yuv420p', r=fps, crf=crf)
        .overwrite_output()
        .run_async(pipe_stdin=True, pipe_stderr=subprocess.PIPE)
    )

    # Create a random message
    msgs = model.get_random_msg()
    with open(output_path.replace(".mp4", ".txt"), "w") as f:
        f.write("".join([str(msg.item()) for msg in msgs[0]]))

    # Process the video
    frame_size = width * height * 3
    chunk = np.zeros((chunk_size, height, width, 3), dtype=np.uint8)
    frame_count = 0
    pbar = tqdm(total=num_frames, desc="Watermark embedding")
    while True:
        in_bytes = process1.stdout.read(frame_size)
        if not in_bytes:
            break
        frame = np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3])
        chunk[frame_count % chunk_size] = frame
        frame_count += 1
        pbar.update(1)
        if frame_count % chunk_size == 0:
            processed_frame = embed_video_clip(model, chunk, msgs)
            process2.stdin.write(processed_frame.tobytes())
    process1.stdout.close()
    process2.stdin.close()
    process1.wait()
    process2.wait()

    return msgs

You are free to upload any video and change the `video_path`.

You can look at the watermark video output in the folder `outputs`.

In [ ]:
# Path to the input video
video_path = "./assets/videos/1.mp4"

# Create the output directory and path
output_dir = "./outputs"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, os.path.basename(video_path))

# Embed the watermark inside the video with a random msg
msgs_ori = embed_video(model, video_path, output_path, 16)
print(f"\nSaved watermarked video to {output_path}")

Watermark embedding: 100%|██████████| 256/256 [03:18<00:00,  1.29it/s]

Saved watermarked video to ./outputs/1.mp4


## Extraction

Load the video output from the embedding process and extract the watermark.

In [ ]:
def detect_video_clip(
    model: Videoseal,
    clip: np.ndarray
) -> torch.Tensor:
    clip_tensor = torch.tensor(clip, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
    outputs = model.detect(clip_tensor, is_video=True)
    output_bits = outputs["preds"][:, 1:]  # exclude the first which may be used for detection
    return output_bits

def detect_video(
    model: Videoseal,
    input_path: str,
    num_frames_for_extraction: int,
    chunk_size: int
) -> None:
    # Read video dimensions
    video_info = get_video_info(input_path)
    width = int(video_info['width'])
    height = int(video_info['height'])
    num_frames = int(video_info['num_frames'])

    # Open the input video
    process1 = (
        ffmpeg
        .input(input_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run_async(pipe_stdout=True, pipe_stderr=subprocess.PIPE)
    )

    # Process the video
    frame_size = width * height * 3
    chunk = np.zeros((chunk_size, height, width, 3), dtype=np.uint8)
    frame_count = 0
    soft_msgs = []
    pbar = tqdm(total=num_frames, desc="Watermark extraction")
    while True:
        if frame_count >= num_frames_for_extraction:
            break
        in_bytes = process1.stdout.read(frame_size)
        if not in_bytes:
            break
        frame = np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3])
        chunk[frame_count % chunk_size] = frame
        frame_count += 1
        pbar.update(1)
        if frame_count % chunk_size == 0:
            soft_msgs.append(detect_video_clip(model, chunk))
    process1.stdout.close()
    process1.wait()

    soft_msgs = torch.cat(soft_msgs, dim=0)
    soft_msgs = soft_msgs.mean(dim=0)  # Average the predictions across all frames
    return soft_msgs

Watermark extraction:  12%|█▎        | 32/256 [00:12<01:30,  2.48it/s]


Binary message extracted with 100.0% bit accuracy


In [ ]:
# Detect the watermark
num_frames_for_extraction = 32
soft_msgs = detect_video(model, output_path, num_frames_for_extraction, 16)
bit_acc = bit_accuracy(soft_msgs, msgs_ori).item() * 100
print(f"\nBinary message extracted with {bit_acc:.1f}% bit accuracy")

## Run other baselines

To download other checkpoints, you can run the following command:

In [ ]:
!pip install huggingface_hub
!huggingface-cli download tangtianzhong/img-wm-torchscript --cache-dir .cache
!mkdir ckpts
!cp .cache/models--tangtianzhong--img-wm-torchscript/snapshots/845dc751783db2a03a4b14ea600b0a4a9aba89aa/*.pt ckpts/
!rm -rf .cache

Fetching 11 files:   0%|                                 | 0/11 [00:00<?, ?it/s]Downloading '.gitattributes' to '.cache/models--tangtianzhong--img-wm-torchscript/blobs/a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'

cin_nsm_decoder.pt:   0%|                            | 0.00/138M [00:00<?, ?B/s]

trustmark_decoder_q.pt:   0%|                       | 0.00/95.3M [00:00<?, ?B/s]


hidden_encoder_48b.pt:   0%|                         | 0.00/755k [00:00<?, ?B/s]



mbrs_256_m256_decoder.pt:   0%|                     | 0.00/81.2M [00:00<?, ?B/s]




mbrs_256_m256_encoder.pt:   0%|                     | 0.00/2.38M [00:00<?, ?B/s]





hidden_decoder_48b.pt:   0%|                        | 0.00/1.21M [00:00<?, ?B/s]






hidden_encoder_48b.pt: 100%|█████████████████| 755k/755k [00:00<00:00, 11.7MB/s]
Download complete. Moving file to .cache/models--tangtianzhong--img-wm-torchscript/blobs/6a7c78241837a455db3d160134fa25e60ec225b548b5d6aab69a30cd3f7b19c3



.gitattributes: 100%|████████████

In [ ]:
from videoseal.utils.cfg import setup_model_from_checkpoint

model = setup_model_from_checkpoint("baseline/trustmark")
model = model.eval()
model = model.to(device)
model.compile()

model.chunk_size = 32  # embed 32 frames/imgs at a time
model.step_size = 4  # propagate the wm to 4 next frame/img
# model.blender.scaling_w *= 1.5  # imperceptibility/robustness trade-off

### Embedding

In [ ]:
# Path to the input video
video_path = "./assets/videos/1.mp4"

# Create the output directory and path
output_dir = "./outputs"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, os.path.basename(video_path))

# Embed the watermark inside the video with a random msg
msgs_ori = embed_video(model, video_path, output_path, 16)
print(f"\nSaved watermarked video to {output_path}")

### Extraction

In [ ]:
# Detect the watermark
num_frames_for_extraction = 32
soft_msgs = detect_video(model, output_path, num_frames_for_extraction, 16)
bit_acc = bit_accuracy(soft_msgs, msgs_ori).item() * 100
print(f"\nBinary message extracted with {bit_acc:.1f}% bit accuracy")